In [ ]:
conda activate genetic_analysis

lets look at the first 4 lines of the fastq in the `data` directory:

In [ ]:
head -4 data/test_sample.fastq

Now lets make this look like a bioinformatics working directory.  
We need to make a `results` directory where all the outputs from the anlysis go.  
Do it now:

In [ ]:
mkdir results

Now we need to populate that with a `qc` (quality control) directory and a `kraken2` (taxonomic assignment) directory:

In [ ]:
mkdir results/qc results/kraken2

Nice that we can do that in a one-liner isn't it?  
  
Let's look at the vital statistics of the fastq file using `seqkit`:

In [ ]:
seqkit stats data/test_sample.fastq

OK, lets qc the fastq using `fastp`:

In [ ]:
fastp \
    --disable_adapter_trimming \
    --in1 data/test_sample.fastq \
    --out1 results/qc/test_sample.qc.fastq \
    -j results/qc/test_sample.json \
    -h results/qc/test_sample.html \
    --qualified_quality_phred 7 \
    --unqualified_percent_limit 40 \
    --average_qual 7 \
    --length_required 1000 \
    --max_len1 1700

Shall we see what that did, `seqkit` to the rescue:

In [ ]:
seqkit stats results/qc/test_sample.qc.fastq

Right, FUCK IT. Unleash the Kraken 2:

In [ ]:
kraken2 \
    results/qc/test_sample.qc.fastq \
    --db bact_db \
    --report results/kraken2/test_sample.txt \
    --output results/kraken2/test_sample.krk \
    --minimum-base-quality 7 \
    --confidence 0.02

Make it more human-readable with a custom python script located in the `scripts` directory:

In [ ]:
python scripts/to_tsv.py -i results/kraken2/test_sample.txt -o results/test_sample.tsv